In [1]:
!pip install pyaudio

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install librosa soundfile

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for audioread: filename=audioread-3.0.0-py3-none-any.whl size=23706 sha256=b9d061b693f87f01af9130d66f83c2cf2abf20ca28df733ea854af0b510ff482
  Stored in directory: c:\users\bharg\appdata\local\pip\cache\wheels\e4\76\a4\cfb55573167a1f5bde7d7a348e95e509c64b2c3e8f921932c3
Successfully built audioread


In [17]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.simplefilter("ignore")

In [4]:

def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
            result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [5]:

emotions={
'01':'neutral',
'02':'calm',
'03':'happy',
'04':'sad',
'05':'angry',
'06':'fearful',
'07':'disgust',
'08':'surprised'}

observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [14]:

def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("SpeechEmotion\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [18]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [19]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [20]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [22]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [23]:
y_pred=model.predict(x_test)

In [29]:
y_pred

array(['happy', 'calm', 'happy', 'happy', 'disgust', 'calm', 'calm',
       'disgust', 'calm', 'happy', 'happy', 'calm', 'fearful', 'happy',
       'disgust', 'happy', 'calm', 'disgust', 'disgust', 'calm', 'calm',
       'disgust', 'disgust', 'calm', 'happy', 'happy', 'happy', 'happy',
       'calm', 'happy', 'happy', 'happy', 'happy', 'fearful', 'happy',
       'calm', 'calm', 'fearful', 'calm', 'disgust', 'happy', 'disgust',
       'calm', 'calm', 'fearful', 'calm', 'disgust', 'calm', 'calm',
       'happy', 'fearful', 'fearful', 'fearful', 'happy', 'happy',
       'happy', 'disgust', 'happy', 'calm', 'calm', 'disgust', 'calm',
       'happy', 'calm', 'happy', 'calm', 'happy', 'disgust', 'fearful',
       'happy', 'happy', 'fearful', 'fearful', 'fearful', 'fearful',
       'disgust', 'fearful', 'happy', 'calm', 'fearful', 'happy', 'calm',
       'fearful', 'calm', 'disgust', 'happy', 'disgust', 'happy', 'happy',
       'happy', 'disgust', 'happy', 'calm', 'happy', 'disgust', 'disgust

In [25]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 68.23%
